In [31]:
import requests

API_URL = "https://realtime-network-intrusion-detection.onrender.com/predict"

flow = {
    "total_bytes": 920,
    "pkt_count": 12,
    "psh_count": 3,
    "fwd_bytes": 67,
    "bwd_bytes": 853,
    "fwd_pkts": 7,
    "bwd_pkts": 5,
    "dport": 80,
    "duration_ms": 98.17028045654297,
    "flow_pkts_per_s": 122.23658671640486,
    "fwd_bwd_ratio": 0.07854630715123095
}

try:
    response = requests.post(API_URL, json=flow, timeout=10)
    response.raise_for_status()  # Vérifie si la réponse contient une erreur HTTP
    print("Réponse de l'API :", response.json())
except requests.exceptions.RequestException as e:
    print("Erreur lors de la requête :", e)

Erreur lors de la requête : 503 Server Error: Service Unavailable for url: https://realtime-network-intrusion-detection.onrender.com/predict


In [29]:
response = requests.post(API_URL, json=flow, timeout=10)

In [30]:
response

<Response [503]>

In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('reported_flows.db')
df = pd.read_sql_query(
    "SELECT * FROM reported_flows ORDER BY reported_at DESC LIMIT 100",
    conn
)
conn.close()

print(df)


   id                            flow_key      src_ip      dst_ip  sport  \
0   3  172.30.0.4:21-172.30.0.6:47830-TCP  172.30.0.6  172.30.0.4  47830   
1   2  172.30.0.4:21-172.30.0.6:47830-TCP  172.30.0.6  172.30.0.4  47830   
2   1  172.30.0.5:80-172.30.0.7:59834-TCP  172.30.0.7  172.30.0.5  59834   

   dport protocol  total_bytes  pkt_count prediction  \
0     21      TCP          295         25              
1     21      TCP          295         25              
2     80      TCP          920         12        Mal   

                                user_id          reported_at  \
0  eea3f842-1049-4b11-b629-aad2c3e56f92  2025-07-06 15:17:00   
1  eea3f842-1049-4b11-b629-aad2c3e56f92  2025-07-06 15:16:55   
2  eea3f842-1049-4b11-b629-aad2c3e56f92  2025-07-06 15:16:46   

                                           flow_data  
0  {"flow_key": "172.30.0.4:21-172.30.0.6:47830-T...  
1  {"flow_key": "172.30.0.4:21-172.30.0.6:47830-T...  
2  {"flow_key": "172.30.0.5:80-172.30.0.7:59834-

In [20]:
df

,id,flow_key,src_ip,dst_ip,sport,dport,protocol,total_bytes,pkt_count,prediction,user_id,reported_at,flow_data
0,3,172.30.0.4:21-172.30.0.6:47830-TCP,172.30.0.6,172.30.0.4,47830,21,TCP,295,25,,eea3f842-1049-4b11-b629-aad2c3e56f92,2025-07-06 15:17:00,"{""flow_key"": ""172.30.0.4:21-172.30.0.6:47830-T..."
1,2,172.30.0.4:21-172.30.0.6:47830-TCP,172.30.0.6,172.30.0.4,47830,21,TCP,295,25,,eea3f842-1049-4b11-b629-aad2c3e56f92,2025-07-06 15:16:55,"{""flow_key"": ""172.30.0.4:21-172.30.0.6:47830-T..."
2,1,172.30.0.5:80-172.30.0.7:59834-TCP,172.30.0.7,172.30.0.5,59834,80,TCP,920,12,Mal,eea3f842-1049-4b11-b629-aad2c3e56f92,2025-07-06 15:16:46,"{""flow_key"": ""172.30.0.5:80-172.30.0.7:59834-T..."


In [24]:
import sqlite3

conn = sqlite3.connect("reported_flows.db")
rows = conn.execute("SELECT count(*) FROM reported_flows").fetchall()
print("Rows:", rows)


Rows: [(3,)]
